In [4]:
from compare_adversarial import *
from xai import latent_space_display_mark

In [6]:
vae = VAE.load("trained_models/VAE")
cnn = load_model("trained_models/CNN/classifier.h5")
xai = xai_model(vae.decoder, cnn, input_shape=(12,))

# input images
# samples_test, sample_labels_test = load_samples_for_test_folder(img_dir='./contrib/DLFuzz/MNIST/seeds_50')
samples_test, sample_labels_test = load_samples_for_test(200)

# prepare
view_samples = samples_test
view_sample_labels = sample_labels_test
# view_samples = samples
# view_sample_labels = sample_labels

x_view = np.reshape(view_samples, (-1, 784))
y_view_onehot = tf.one_hot(tf.constant(view_sample_labels), depth=10).numpy()
h_view = vae.encoder.predict(x_view)

K.set_learning_phase(0)
dlfuzz = DLFuzz(cnn)

7/7 [==============================] - 0s 2ms/step


/opt/anaconda3/envs/bachelor/lib/python3.9/site-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [12]:
# start
for i in tqdm(range(len(x_view))):

    # calculate fuzz image
    image_org = np.array([view_samples[i]], dtype="float32")
    label_org = view_sample_labels[i]
    image_gen_fuzz = dlfuzz.generate_adversarial_image(image_org)
    print(image_gen_fuzz.shape)
    # of shape (28, 28, 1)

    # calculate latent variant image
    x = np.array([h_view[i]])
    y = np.array([y_view_onehot[i]])
    g = gradient_of_x(x, y, xai)
    g_npy = np.squeeze(g.numpy())
    
    # var = 0.5
    # h_lava = get_h_lava_via_one_step(x, g_npy, step=var)
    # image_gen_lava = vae.decoder.predict(h_lava)[0].reshape((28, 28, 1))

    alpha = 0.01  # Learning rate
    num_iterations = 100  # Number of iterations for gradient descent
    [image_gen_lava, prediction] = gradient_descent_generate(h_view[i], y_view_onehot[i], alpha, num_iterations)

    label_fuzz = np.argmax(cnn.predict(np.array([image_gen_fuzz]))[0])
    label_lava = np.argmax(cnn.predict(np.array([image_gen_lava]))[0])

    # List of images and their titles
    images = [image_org, image_gen_fuzz, image_gen_lava]
    titles = [f'image_org_{label_org}', f'image_gen_fuzz_{label_fuzz}', f'image_gen_lava_{label_lava}']

    # Plot the images
    if label_lava != label_org:
        plot_image_comparison(images, titles)

        # in latent space
        # Euclidean distance (L2 norm) 
        h_fuzz = vae.encoder.predict(np.array([image_gen_fuzz.reshape((784,))]))[0]
        d_fuzz = np.linalg.norm(x - h_fuzz)
        d_lava = np.linalg.norm(x - h_lava)
        print(f"d_fuzz: {d_fuzz}\nd_lava: {d_lava}")

        # in image space
        kl_fuzz = kl_divergence(image_org, image_gen_fuzz)
        kl_lava = kl_divergence(image_org, image_gen_lava)
        print(f"kl_fuzz: {kl_fuzz}\nkl_lava: {kl_lava}")

        ws_fuzz = ws_distance(image_org, image_gen_fuzz)
        ws_lava = ws_distance(image_org, image_gen_lava)
        print(f"ws_fuzz: {ws_fuzz}\nws_lava: {ws_lava}")

        js_fuzz = js_divergence(image_org, image_gen_fuzz)
        js_lava = js_divergence(image_org, image_gen_lava)
        print(f"js_fuzz: {js_fuzz}\njs_lava: {js_lava}")

        ce_fuzz = cross_entropy(image_org, image_gen_fuzz)
        ce_lava = cross_entropy(image_org, image_gen_lava)
        print(f"ce_fuzz: {ce_fuzz}\nce_lava: {ce_lava}")

        mse_fuzz = mse_loss(image_org, image_gen_fuzz)
        mse_lava = mse_loss(image_org, image_gen_lava)
        print(f"mse_fuzz: {mse_fuzz}\nmse_lava: {mse_lava}")
        

  0%|                                                   | 0/200 [00:00<?, ?it/s]

1/1 [==============================] - 0s 20ms/step


  0%|                                                   | 0/200 [00:02<?, ?it/s]

(28, 28, 1)


NameError: name 'gradient_of_x' is not defined